In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random
import calendar
import numpy as np
import sys

In [ ]:
# Id và địa điểm
IdAddress = [[3726177, 'Phú Quốc'],
            [3712045, 'Đà Lạt'],
            [3727579, 'Quy Nhơn'],
            [3733750, 'Vũng Tàu'],
            [3723998, 'Nha Trang'],
            [3712125, 'Đà Nẵng'],
            [3725092, 'Phan Thiết'],
            [3715887, 'Huế'],
            [3715584, 'Hội An']]
            # [3730078, 'TP. Hồ Chí Minh']

In [ ]:
def crawl_link(Id = 0, Address = '_'):
  cur_tab = 0
  tab = 1
  list_NumberRV = []
  list_HotelId = []
  list_HotelName = []

  while cur_tab < tab:
    url = 'https://www.booking.com/searchresults.vi.html?&dest_id=-' + str(Id) + '&dest_type=city&offset=' + str(cur_tab*25)

    headers = {
        "Connection": "keep-alive",
        "Host": "www.booking.com",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36",
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
    }

    req_data = requests.post(url, headers = headers).content.decode("utf-8")
    soup_add = BeautifulSoup(req_data, 'html.parser')

    if cur_tab == 0:
      CountHotel = soup_add.find('h1', {'aria-live':"assertive"}).text[:-9].split()[-1].replace('.', '')
      tab = int(float(CountHotel)/25)

    cur_tab +=1

    for info_Hotel in soup_add.find_all('div', {'data-testid':'property-card'}):
      link = info_Hotel.find('a', {'data-testid':'property-card-desktop-single-image'})['href']
      list_HotelId.append(link[33 : link.find(".vi.html")])
      list_HotelName.append(info_Hotel.text[:info_Hotel.text.find("Mở trong cửa sổ mới")])
      try:
        list_NumberRV.append(int(info_Hotel.find('div', {'data-testid':'review-score'}).text[:-9].split()[-1].replace('.', '')))
      except:
        list_NumberRV.append(0)

  df_link = pd.DataFrame({'Address': Address, 'HotelName': list_HotelName, 'HotelId':list_HotelId, 'NumberRV': list_NumberRV})
  df_link = df_link.drop_duplicates(subset='HotelId', keep='first')
  return df_link

In [ ]:
def df_Review(Address = '_', HotelName = '_', HotelId = '_', NumberRV = 25):
  list_UserName = []
  list_Nation = []
  list_Ratting = []
  list_Date = []
  list_Review = []
  cur_tab = 0
  while cur_tab < NumberRV/25:
    url = 'https://www.booking.com/reviewlist.vi.html?aid=304142&cc1=vn;pagename=' + HotelId + '&&offset=' + str(cur_tab*25) + ';rows=25'

    headers = {
        "Connection": "keep-alive",
        "Host": "www.booking.com",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36",
        "Accept": '/',
    }

    req_data = requests.post(url, headers = headers).content.decode("utf-8")
    soup_hotel = BeautifulSoup(req_data, 'html.parser')

    cur_tab += 1

    for info_user in soup_hotel.find_all('div', {'class':'c-review-block'}):

      review = ''
      for cmt in info_user.find_all('span', {'class':'c-review__body'}):
        review += cmt.text + '.'

      if review == 'Không có bình luận nào cho đánh giá này.':
        cur_tab = NumberRV/25 + 1
        break
      list_Review.append(review)

      list_UserName.append(info_user.find('span', {'class':'bui-avatar-block__title'}).text)
      try:
        list_Nation.append(info_user.find('span', {'class':'bui-avatar-block__subtitle'}).text.replace('\n', ''))
      except:
        list_Nation.append('Unknown')
      list_Ratting.append(info_user.find('div', {'class':'bui-review-score__badge'}).text.replace(',', '.'))

      def random_day_in_month(month, year):
        _, num_days = calendar.monthrange(year, month)
        random_day = random.randint(1, num_days)
        return random_day

      temp_date = info_user.find('span', {'class':'c-review-block__date'}).text.replace('tháng ', '/').replace('\n', '')
      try:
        date_split = temp_date.split("/")
        date_split[0] = random_day_in_month(int(date_split[1]), int(date_split[2]))
        list_Date.append(str(date_split[0]) + '/' + date_split[1] + '/' + date_split[2])
      except:
        list_Date.append(temp_date)

  df_reviewHotel = pd.DataFrame({'CustomerName':list_UserName, 'Address':Address, 'NameHotel': HotelName, 'Ratting': list_Ratting, 'Review': list_Review, 'Date': list_Date, 'Nation': list_Nation})
  return df_reviewHotel

In [ ]:
for k in range(len(IdAddress)):
  df_link = crawl_link(IdAddress[k][0], IdAddress[k][1])
  df_link = df_link.sort_values('NumberRV', ascending=False)[:20]
  df_link = df_link.set_index(np.array(range(df_link.shape[0])))

  print(str(k+1) + ':' + IdAddress[k][1])
  for i in range(len(df_link)):
    print(' Số khách sạn đã thu thập:', i, end="")
    sys.stdout.flush()

    df_temp = df_Review(df_link['Address'][i], df_link['HotelName'][i], df_link['HotelId'][i], df_link['NumberRV'][i])
    if i == 0: df_location = df_temp
    else: df_location = pd.concat([df_location, df_temp])

    print("\r", end="")
    sys.stdout.flush()
  df_location = df_location.set_index(np.array(range(df_location.shape[0])))

  if k == 0: df_total = df_location
  else: df_total = pd.concat([df_total, df_location])

df_total.drop_duplicates(inplace=True)

1:Phú Quốc
2:Đà Lạt
3:Quy Nhơn
4:Vũng Tàu
5:Nha Trang
6:Đà Nẵng
7:Phan Thiết
8:Huế
9:Hội An


In [ ]:
df_booking = df_total.copy()

In [ ]:
df_booking = df_booking[df_booking.Nation == 'Việt Nam']

In [ ]:
df_booking = df_booking.set_index(np.array(range(df_booking.shape[0])))

In [ ]:
df_booking = df_booking.drop(index=72)

In [ ]:
df_booking.iloc[253].Review

'Thái độ phục vụ của NV nhà hàng.Đồ ăn sáng ít.View & vật dụng trong phòng chưa đc như mong đội.'

In [ ]:
df_booking[250:255]

,CustomerName,Address,NameHotel,Ratting,Review,Date,Nation
250,Vy,Phú Quốc,SOL by Meliá Phu Quoc,2.0,Mình đã có một kỳ nghỉ ngơi rất dễ thương tại ...,25/4/2021,Việt Nam
251,Thành,Phú Quốc,SOL by Meliá Phu Quoc,10,Bãi biển.Ăn sáng ko đa dạng món.,25/4/2021,Việt Nam
252,Thi,Phú Quốc,SOL by Meliá Phu Quoc,1.0,Phòng đẹp.Thái độ. Dịch vụ không chuyên nghiệp..,22/4/2021,Việt Nam
253,Hoang,Phú Quốc,SOL by Meliá Phu Quoc,6.0,Thái độ phục vụ của NV nhà hàng.Đồ ăn sáng ít....,26/4/2021,Việt Nam
254,Lonh,Phú Quốc,SOL by Meliá Phu Quoc,10,Phòng rộng view đepj sạch sẽ.Gối hơi cao nhà t...,27/4/2021,Việt Nam


In [ ]:
df_booking.to_csv('data_raw_booking.csv', index=False, header=True)

In [ ]:
df_check = pd.read_csv('data_raw_booking.csv')

In [ ]:
df_check.Address.value_counts()

Vũng Tàu      6317
Đà Lạt        4364
Nha Trang     3475
Đà Nẵng       2918
Quy Nhơn      1982
Huế           1846
Phú Quốc      1678
Phan Thiết    1175
Hội An        1149
Name: Address, dtype: int64

In [ ]:
df_booking = df_total[df_total.Nation == 'Việt Nam']
df_booking['Review'] = df_booking['Review'].str.replace('\n', '.')
df_booking['Review'] = df_booking['Review'].str.replace('\r', '.')
df_booking.drop_duplicates(inplace=True)

In [ ]:
df_booking = df_booking.drop_duplicates(subset='Review', keep='first')

In [ ]:
import regex as re

uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"


def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic


dicchar = loaddicchar()

# Đưa toàn bộ dữ liệu qua hàm này để chuẩn hóa lại
def convert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

def no_accent_vietnamese(s):
    convert_unicode(s)
    s = re.sub(u'[àáạảãâầấậẩẫăằắặẳẵ]', 'a', s)
    s = re.sub(u'[ÀÁẠẢÃĂẰẮẶẲẴÂẦẤẬẨẪ]', 'A', s)
    s = re.sub(u'[èéẹẹẻẽêềếệểễ]', 'e', s)
    s = re.sub(u'[ÈÉẸẺẼÊỀẾỆỂỄ]', 'E', s)
    s = re.sub(u'[òóọỏõôồốộổỗơờớợởỡ]', 'o', s)
    s = re.sub(u'[ÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠ]', 'O', s)
    s = re.sub(u'[ìíịỉĩ]', 'i', s)
    s = re.sub(u'[ÌÍỊỈĨ]', 'I', s)
    s = re.sub(u'[ùúụủũưừứựửữ]', 'u', s)
    s = re.sub(u'[ƯỪỨỰỬỮÙÚỤỦŨ]', 'U', s)
    s = re.sub(u'[ỳýỵỷỹ]', 'y', s)
    s = re.sub(u'[ỲÝỴỶỸ]', 'Y', s)
    s = re.sub(u'Đ', 'D', s)
    s = re.sub(u'đ', 'd', s)

    return s

In [ ]:
bang_nguyen_am = [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]
bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']

In [ ]:
nguyen_am_to_ids = {}

for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)

In [ ]:
def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True

In [ ]:
def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word

    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word

    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            return ''.join(chars)

    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
        else:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    return ''.join(chars)

In [ ]:
#Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
def chuan_hoa_dau_cau_tieng_viet(sentence):
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)

In [ ]:
#Chuẩn hóa viết tắt
def chuan_hoa_viet_tat_tieng_viet(sentence):
  sentence = re.sub(r'\b[k][s]([a][n]|[ạ][n])?\b', 'khách sạn', sentence)
  sentence = re.sub(r'\b[n][v]([i][e][n]|[i][ê][n])?\b', 'nhân viên', sentence)
  sentence = re.sub(r'\b[d][v]([u]|[ụ])?\b', 'dịch vụ', sentence)
  sentence = re.sub(r'\b[v][i][e][w]\b', 'khung cảnh', sentence)
  sentence = re.sub(r'\b[b][o][o][k]\b', 'đặt', sentence)
  sentence = re.sub(r'\b[g]([d]|[đ])\b', 'gia đình', sentence)
  sentence = sentence.replace('booking', 'đặt phòng')
  sentence = sentence.replace('set up', 'thiết kế')
  sentence = sentence.replace('setup', 'thiết kế')
  sentence = sentence.replace('set', 'bộ')
  sentence = sentence.replace('support', 'hỗ trợ')
  sentence = sentence.replace('check out', 'trả phòng')
  sentence = sentence.replace('checkout', 'trả phòng')
  sentence = sentence.replace('check in', 'nhận phòng')
  sentence = sentence.replace('checkin', 'nhận phòng')
  sentence = sentence.replace('delay', 'trì hoãn')
  sentence = sentence.replace('resort', 'khu nghĩ dưỡng')
  sentence = sentence.replace('service', 'dịch vụ')
  sentence = sentence.replace('menu', 'thực đơn')
  sentence = sentence.replace('upgrade', 'nâng cấp')
  sentence = sentence.replace('take care', 'chăm sóc')
  sentence = sentence.replace('takecare', 'chăm sóc')
  sentence = sentence.replace('thank you', 'cảm ơn')

  sentence = re.sub(r'\b[t][h][a][n][k][s]?\b', 'cảm ơn', sentence)
  sentence = re.sub(r'\b[đ]\b', 'điểm', sentence)
  sentence = re.sub(r'\b([d]|[đ])[c]\b', 'được', sentence)
  sentence = re.sub(r'\b[o][k]\b', 'tốt', sentence)
  sentence = re.sub(r'\b[v][s]\b', 'với', sentence)
  sentence = re.sub(r'\b[k][o]?\b', 'không', sentence)
  sentence = re.sub(r'\b[k][h]\b', 'không', sentence)
  sentence = re.sub(r'\b[n][i][c][e]?\b', 'tuyệt vời', sentence)
  # nma, j, mn, nhah
  return sentence

In [ ]:
# BEFORE LABEL
def text_preprocess_before_label(document):
    # chuẩn hóa unicode
    document = convert_unicode(document)
    # chuẩn hóa cách gõ dấu tiếng Việt
    document = chuan_hoa_dau_cau_tieng_viet(document)
    # đưa về lower (chữ thường)
    # document = document.lower()
    # xóa các ký tự không cần thiết
    document = re.sub(r'[&]', 'và', document)
    document = re.sub(r'[^\s\w%!?.,]', ' ', document)
    # xóa khoảng trắng thừa
    document = re.sub(r'\s+', ' ', document).strip()
    # Sửa chính tả, lỗi cú pháp, viết tắt, tiếng anh lẫn vào, không dấu bằng tay trên gg sheet
    document = chuan_hoa_viet_tat_tieng_viet(document)
    return document

In [ ]:
df_total_new = df_check.copy()

In [ ]:
for i in range(len(df_total_new)):
  df_total_new['Review'][i] = text_preprocess_before_label(df_total_new['Review'][i])
  df_total_new['Review'][i] = re.sub(r'\b[h]\b', 'giờ', df_total_new['Review'][i])

<ipython-input-26-e30b3d6e2f19>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total_new['Review'][i] = text_preprocess_before_label(df_total_new['Review'][i])
<ipython-input-26-e30b3d6e2f19>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total_new['Review'][i] = re.sub(r'\b[h]\b', 'giờ', df_total_new['Review'][i])


In [ ]:
df_total_new.drop('Nation', axis=1, inplace=True)

In [ ]:
df_total_new

,CustomerName,Address,NameHotel,Ratting,Review,Date
0,Thanh,Phú Quốc,Ocean Bay Phu Quoc Resort and Spa,9.0,bữa sáng có nhiều sự lựa trọn và phù hợp khẩu ...,8/9/2023
1,Đào,Phú Quốc,Ocean Bay Phu Quoc Resort and Spa,9.0,mọi thứ đều tốt.nhân viên lái xe đưa khách ra ...,17/8/2023
2,Dang,Phú Quốc,Ocean Bay Phu Quoc Resort and Spa,10.0,"phòng óc sạch sẽ, nhân viên nhiệt tình và rất ...",23/7/2023
3,Nhon,Phú Quốc,Ocean Bay Phu Quoc Resort and Spa,10.0,nhân viên rất thân thiện.không có.,2/7/2023
4,Lê,Phú Quốc,Ocean Bay Phu Quoc Resort and Spa,9.0,"buffet ngon, nhân viên thân thiện.vòi nước nón...",10/7/2023
...,...,...,...,...,...,...
24899,Phạm,Hội An,Blooms Boutique Village,8.0,"bữa sáng tốt, nếu thêm món ăn thì sẽ tuyệt vời...",2/9/2022
24900,Mee,Hội An,Blooms Boutique Village,8.0,khu vực rất thanh bình.buổi sáng muộn hơn giờ ...,10/8/2022
24901,Anh,Hội An,Blooms Boutique Village,9.0,"nhân viên nhiệt tình, lễ phép và lịch sự. bữa ...",25/8/2022
24902,Quỳnh,Hội An,Blooms Boutique Village,10.0,"không gian rộng, thoáng mát, rất sạch sẽ. nhân...",14/8/2022


In [ ]:
df_booking = df_total_new.copy()

In [ ]:
df_booking.sample(frac=1).reset_index(drop=True)

In [ ]:
list_Hotel = df_booking.NameHotel.value_counts().index
df_booking_new = df_booking[:1]

for name_Hotel in list_Hotel:
  df_temp = df_booking[df_booking.NameHotel==name_Hotel]
  df_temp = df_temp.sample(frac=1).reset_index(drop=True)
  df_temp_50 = df_temp[df_temp.Review.str.len() >= 70]

  num = df_temp[df_temp.Ratting < 8].Ratting.value_counts().sum()
  if num < 5: num = 5
  df_temp_50 = df_temp_50[df_temp_50.Ratting >= 8][:num-2]

  df_booking_new = pd.concat([df_booking_new, df_temp_50, df_temp[df_temp.Ratting < 8]])
  df_booking_new = df_booking_new[df_booking_new.Review.str.len() <= 350]
  df_booking_new = df_booking_new[df_booking_new.Review.str.len() >= 30]

In [ ]:
df_booking_new[df_booking_new.Ratting < 8].Ratting.value_counts().sum()

3742

In [ ]:
df_booking_new[df_booking_new.Ratting >= 8].Ratting.value_counts().sum()

3858

In [ ]:
df_booking_new.Date = pd.to_datetime(df_booking_new['Date'])
df_booking_new = df_booking_new.sort_values('Date')

<ipython-input-96-d84860da19c4>:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_booking_new.Date = pd.to_datetime(df_booking_new['Date'])


In [ ]:
df_booking_new

,CustomerName,Address,NameHotel,Ratting,Review,Date
60,Đặng,Quy Nhơn,La Cactus Hotel,10.0,trên cả tuyệt vời khách sạn vô cùng đáng lưu t...,2020-01-10
44,Cao,Quy Nhơn,Mira Hotel Quy Nhơn,9.0,"khách sạn sạch sẽ, thoáng mát. nhân viên thân ...",2020-01-10
35,Trang,Đà Lạt,Redhouse Dalat Hotel,8.0,"nhân viên và chị chủ khách sạn khá thân thiện,...",2020-01-10
45,Anh,Vũng Tàu,Pullman Vung Tau,8.0,"yên tĩnh, khung cảnh biển đẹp, giường nệm dễ c...",2020-01-10
167,Chuột,Huế,The Sunriver Boutique Hotel Hue,5.0,vị trí gần trung tâm..mình ở 2 đêm nhưng khách...,2020-01-10
...,...,...,...,...,...,...
13,Thưởng,Nha Trang,Aaron Hotel,10.0,"phòng sạch sẽ , nhân viên thân thiện bạn nữ lễ...",2023-12-08
2,Dang,Đà Nẵng,Sekong Hotel Da Nang,7.0,"gần biển, sạch sẽ.đối diện bị khu bella maison...",2023-12-08
293,Nguyễn,Vũng Tàu,Vung Tau Melody Apartment,4.0,"phòng không được khang trang như hình, nội thấ...",2023-12-08
0,Ngoclan,Huế,Hue Sweethouse 1 Homestay,10.0,"yên tĩnh, tiện nghi, nhân viên nhẹ nhàng lịch ...",2023-12-09


In [ ]:
df_booking_new.to_csv('data_raw_booking_pro.csv', index=False, header=True)

In [ ]:
df_booking_new[df_booking_new.Review.str.len() <= 500].Ratting.value_counts()

10.0    2090
7.0     1906
9.0     1311
8.0     1250
5.0      916
6.0      778
1.0      409
4.0      292
3.0      207
2.0       86
7.5       11
9.6        7
7.1        6
7.9        5
6.2        4
8.8        4
9.2        3
2.5        2
6.7        2
8.3        2
5.4        2
5.8        1
3.3        1
Name: Ratting, dtype: int64